In [1]:
!pip install tensorflow librosa soundfile --quiet


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import tensorflow as tf
model = tf.keras.models.load_model("C:/Users/awsmd/OneDrive/Documents/GitHub/Deepfake-Detection/deepfake_detection.h5",compile=False)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 109, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 126, 107, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 53, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 51, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 25, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 48000)             0     

In [2]:
import librosa
import numpy as np

N_MELS      = 128         
N_FRAMES    = 109        
SAMPLE_RATE = 16000      
DURATION    = N_FRAMES * 512 / SAMPLE_RATE   # 512 = hop_length default


def preprocess_audio(path, sr=SAMPLE_RATE):
    import librosa, numpy as np


    y, _ = librosa.load(path, sr=sr)

    target_len = int(DURATION * sr)
    if len(y) < target_len:
        y = np.pad(y, (0, target_len - len(y)))
    else:
        y = y[:target_len]

    # mel‑spectrogram -> log‑power
    mel = librosa.feature.melspectrogram(
        y=y,
        sr=sr,
        n_mels=N_MELS,
        hop_length=512,        # must equal training hop_length
        n_fft=1024             # usually 2× hop_length but depends on training
    )
    mel_db = librosa.power_to_db(mel, ref=np.max)

    mel_db = mel_db[:, :N_FRAMES]        # trim if 110
    if mel_db.shape[1] < N_FRAMES:       # pad if 108
        mel_db = np.pad(mel_db, ((0,0),(0, N_FRAMES-mel_db.shape[1])))

    mel_db = mel_db[..., np.newaxis]         # (128,109,1)
    return np.expand_dims(mel_db, 0)         # (1,128,109,1)


CLASS_NAMES = ['bonafide', 'spoof']

In [3]:
test_path = 'C:/Users/awsmd/OneDrive/Documents/GitHub/Deepfake-Detection/AUDIO/REAL/biden-original.wav'
x = preprocess_audio(test_path)
probs = model.predict(x)[0]
print({c: float(p) for c, p in zip(CLASS_NAMES, probs)})
print('Predicted label:', CLASS_NAMES[int(np.argmax(probs))])

1/1 [==============================] - 5s 5s/step
{'bonafide': 1.0, 'spoof': 2.282783519653603e-19}
Predicted label: bonafide


In [5]:
from pathlib import Path

audio_dir = Path('C:/Users/awsmd/OneDrive/Documents/GitHub/Deepfake-Detection/AUDIO/REAL')
results = {}
for wav in audio_dir.glob('*.wav'):
    x = preprocess_audio(wav)
    probs = model.predict(x)[0]
    results[wav.name] = {
        'bonafide_prob': float(probs[0]),
        'spoof_prob': float(probs[1]),
        'predicted': CLASS_NAMES[int(np.argmax(probs))]
    }
print(results)

1/1 [==============================] - 0s 15ms/step
{'biden-original.wav': {'bonafide_prob': 1.0, 'spoof_prob': 2.282783519653603e-19, 'predicted': 'bonafide'}, 'linus-original.wav': {'bonafide_prob': 1.0, 'spoof_prob': 3.1079656890757113e-15, 'predicted': 'bonafide'}, 'margot-original.wav': {'bonafide_prob': 1.0, 'spoof_prob': 2.1710851781269328e-29, 'predicted': 'bonafide'}, 'musk-original.wav': {'bonafide_prob': 1.0, 'spoof_prob': 1.6805403665277256e-15, 'predicted': 'bonafide'}, 'obama-original.wav': {'bonafide_prob': 1.0, 'spoof_prob': 1.5827626311449034e-29, 'predicted': 'bonafide'}, 'ryan-original.wav': {'bonafide_prob': 1.0, 'spoof_prob': 1.494848879004066e-19, 'predicted': 'bonafide'}, 'taylor-original.wav': {'bonafide_prob': 1.0, 'spoof_prob': 1.36583549693289e-25, 'predicted': 'bonafide'}, 'trump-original.wav': {'bonafide_prob': 1.0, 'spoof_prob': 6.162955607478458e-25, 'predicted': 'bonafide'}}
